## Trying the model on images from the supervisor file

In [ ]:
# --- KONFIGURACJA ---
# Lista Twoich folderów, gdzie masz wymieszane zdjęcia i txt
source_folders = [
    r"c:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\cams_nov\sprogoe\2025-11-22",
    r"c:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\cams_nov\storebaelt_east\2025-11-22"
]

# Gdzie ma powstać nowy, uporządkowany dataset?
dest_root = r"c:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\dataset_fixed"

# --------------------

def prepare_data():
    img_dest = Path(dest_root) / "images" / "val"
    lbl_dest = Path(dest_root) / "labels" / "val"

    # Tworzymy foldery
    img_dest.mkdir(parents=True, exist_ok=True)
    lbl_dest.mkdir(parents=True, exist_ok=True)

    print(f"Tworzenie datasetu w: {dest_root}")

    total_imgs = 0
    total_lbls = 0

    for src in source_folders:
        src_path = Path(src)
        if not src_path.exists():
            print(f"BŁĄD: Folder nie istnieje: {src}")
            continue

        print(f"Przetwarzanie: {src}...")
        
        # Pobieramy wszystkie pliki
        files = list(src_path.glob("*"))
        
        for f in files:
            if f.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                # Kopiuj zdjęcie
                shutil.copy2(f, img_dest / f.name)
                total_imgs += 1
            elif f.suffix.lower() == '.txt':
                # Kopiuj etykietę
                shutil.copy2(f, lbl_dest / f.name)
                total_lbls += 1

    print("-" * 30)
    print(f"Gotowe!")
    print(f"Skopiowano zdjęć: {total_imgs}")
    print(f"Skopiowano etykiet: {total_lbls}")
    
    # Tworzenie nowego pliku YAML
    yaml_content = f"""
path: {dest_root}
train: images/val  # Używamy val też jako train, jeśli chcesz tylko testować
val: images/val
names:
  0: ship
"""
    yaml_path = Path(dest_root) / "data_fixed.yaml"
    with open(yaml_path, "w") as f:
        f.write(yaml_content)
    
    print(f"\nStworzono nowy plik konfiguracyjny: {yaml_path}")
    print("Użyj tego pliku do walidacji!")

if __name__ == "__main__":
    prepare_data()

In [ ]:
model = YOLO("C:\\Users\\szymo\\Desktop\\DTU\\3rd_semester\\Individual_project_demo\\runs\\detect\\finetune_night_ships\\weights\\best.pt")

# Wskazujemy na nowy plik YAML stworzony przez skrypt wyżej
model.val(data=r"c:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\dataset_only_ships\data_only_ships.yaml", split='val')

In [ ]:
# --- KONFIGURACJA ---
source_root = Path(r"c:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\dataset_fixed")
dest_root = Path(r"c:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\dataset_split_70_20_10")
# --------------------

def split_and_filter_dataset():
    # 1. Czyszczenie folderu docelowego (żeby nie mieszać ze starymi próbami)
    if dest_root.exists():
        print("Usuwam stary folder docelowy, żeby zrobić czysty podział...")
        shutil.rmtree(dest_root)
    
    # 2. Zbieranie wszystkich poprawnych par (zdjęcie + label ze statkiem)
    print("Zbieranie danych z folderu źródłowego...")
    src_labels_dir = source_root / "labels"
    src_images_dir = source_root / "images"
    
    valid_pairs = [] # Lista tupli: (ścieżka_do_label, ścieżka_do_img)

    # Szukamy wszystkich plików .txt w źródle (rekurencyjnie, nieważne czy są w train/val/test)
    for label_file in src_labels_dir.rglob("*.txt"):
        
        # Sprawdzamy czy plik NIE jest pusty (czy ma statek)
        has_content = False
        with open(label_file, 'r') as f:
            if any(line.strip() for line in f):
                has_content = True
        
        if has_content:
            # Szukamy odpowiadającego zdjęcia
            # Musimy zgadnąć gdzie jest zdjęcie, bo struktura folderów images zazwyczaj odpowiada labels
            # Sztuczka: podmieniamy w ścieżce "labels" na "images" i szukamy pliku
            
            # Pobieramy relatywną ścieżkę pliku txt względem folderu labels
            # np. jeśli plik to .../labels/train/plik.txt, to relative to "train/plik.txt"
            rel_path = label_file.relative_to(src_labels_dir)
            
            image_found = None
            # Szukamy zdjęcia w analogicznym podfolderze w images
            for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
                possible_img_path = src_images_dir / rel_path.with_suffix(ext)
                if possible_img_path.exists():
                    image_found = possible_img_path
                    break
            
            if image_found:
                valid_pairs.append((label_file, image_found))

    print(f"Znaleziono łącznie {len(valid_pairs)} zdjęć ze statkami.")
    
    if len(valid_pairs) == 0:
        print("Błąd: Nie znaleziono żadnych danych! Sprawdź ścieżki.")
        return

    # 3. Mieszanie i dzielenie (70% / 20% / 10%)
    random.seed(42) # Żeby zawsze dzieliło tak samo
    random.shuffle(valid_pairs)

    total = len(valid_pairs)
    train_end = int(total * 0.70)
    val_end = int(total * 0.90) # 70% + 20% = 90%

    train_set = valid_pairs[:train_end]
    val_set = valid_pairs[train_end:val_end]
    test_set = valid_pairs[val_end:]

    print(f"Podział: Train: {len(train_set)}, Val: {len(val_set)}, Test: {len(test_set)}")

    # 4. Funkcja kopiująca
    def copy_files(dataset, split_name):
        dst_img_dir = dest_root / "images" / split_name
        dst_lbl_dir = dest_root / "labels" / split_name
        dst_img_dir.mkdir(parents=True, exist_ok=True)
        dst_lbl_dir.mkdir(parents=True, exist_ok=True)

        for lbl_src, img_src in dataset:
            shutil.copy2(lbl_src, dst_lbl_dir / lbl_src.name)
            shutil.copy2(img_src, dst_img_dir / img_src.name)

    # Kopiujemy fizycznie pliki
    print("Kopiowanie plików (to chwilę potrwa)...")
    copy_files(train_set, "train")
    copy_files(val_set, "val")
    copy_files(test_set, "test")

    # 5. Tworzenie YAML
    yaml_content = f"""
path: {dest_root.as_posix()}
train: images/train
val: images/val
test: images/test

names:
  0: ship
"""
    yaml_path = dest_root / "data_split.yaml"
    with open(yaml_path, "w") as f:
        f.write(yaml_content)

    print("-" * 30)
    print("GOTOWE!")
    print(f"Nowy dataset jest tu: {dest_root}")
    print(f"Plik YAML do treningu: {yaml_path}")

if __name__ == "__main__":
    split_and_filter_dataset()

In [ ]:
# --- KONFIGURACJA ---
json_file = 'cams_nov/_annotations.coco.json'
image_folders = [
    'cams_nov/sprogoe/2025-11-22', 
    'cams_nov/storebaelt_east/2025-11-22'
] 
# --------------------

def normalize_name(name):
    """
    Czyści nazwę pliku do gołego ciągu znaków (tylko litery i cyfry).
    Naprawia problem Roboflow z doklejaniem '_jpg' do nazwy.
    """
    name = os.path.basename(name)
    
    # 1. Usuwamy hash Roboflow (wszystko po .rf.)
    name = re.split(r'\.rf\.', name)[0]
    
    # 2. Usuwamy prawdziwe rozszerzenie
    name = os.path.splitext(name)[0]
    
    # 3. KLUCZOWA POPRAWKA: Usuwamy doklejone rozszerzenia w nazwie (np. _jpg, _jpeg)
    # Flaga IGNORECASE pozwala łapać JPG, jpg, Jpg itd.
    name = re.sub(r'(_jpg|_jpeg|_png)$', '', name, flags=re.IGNORECASE)
    name = re.sub(r'(jpg|jpeg|png)$', '', name, flags=re.IGNORECASE)

    # 4. Zamieniamy wszystko co nie jest cyfrą lub literą na pusty ciąg
    # Usuwamy myślniki, dwukropki, podkreślniki, kropki
    clean_name = re.sub(r'[^a-zA-Z0-9]', '', name)
    
    return clean_name

def convert_coco_to_yolo_smart():
    print(f"Katalog roboczy skryptu: {os.getcwd()}")
    
    # 1. Skanowanie dysku
    print("Skanowanie plików na dysku...")
    disk_file_map = {} 
    
    total_files_found = 0
    for folder in image_folders:
        if not os.path.exists(folder):
            print(f"❌ BŁĄD: Folder nie istnieje: '{folder}'")
            continue
            
        print(f"   Przeszukiwanie: {folder}...")
        for f in os.listdir(folder):
            if f.lower().endswith(('.jpg', '.jpeg', '.png')):
                norm = normalize_name(f)
                full_path = os.path.join(folder, f)
                disk_file_map[norm] = full_path
                total_files_found += 1
    
    print(f"✅ Znaleziono łącznie {total_files_found} zdjęć w folderach.")
    if total_files_found == 0:
        print("❌ BŁĄD KRYTYCZNY: Nie znaleziono żadnych zdjęć.")
        return

    # 2. Wczytujemy JSON
    if not os.path.exists(json_file):
        print(f"❌ BŁĄD: Nie znaleziono pliku adnotacji: {json_file}")
        return

    try:
        with open(json_file, 'r') as f:
            data = json.load(f)
    except Exception as e:
        print(f"❌ BŁĄD przy otwieraniu JSON: {e}")
        return

    # Mapowanie kategorii
    categories = {cat['id']: cat['name'] for cat in data['categories']}
    sorted_ids = sorted(categories.keys())
    id_map = {original_id: new_id for new_id, original_id in enumerate(sorted_ids)}
    
    print("\nMapowanie klas (COCO -> YOLO):")
    for original, new in id_map.items():
        print(f"  ID {original} ('{categories[original]}') -> ID {new}")

    # 3. Generowanie
    print("\nRozpoczynam dopasowywanie i tworzenie .txt...")
    
    images_info = {img['id']: img for img in data['images']}
    
    annotations_by_image = {}
    for ann in data['annotations']:
        img_id = ann['image_id']
        if img_id not in annotations_by_image:
            annotations_by_image[img_id] = []
        annotations_by_image[img_id].append(ann)

    processed_count = 0
    matched_count = 0
    
    for img_id, anns in annotations_by_image.items():
        img_data = images_info[img_id]
        json_filename = img_data['file_name']
        
        # Normalizacja nazwy z JSON
        json_norm = normalize_name(json_filename)
        
        # PRÓBA DOPASOWANIA
        real_path = disk_file_map.get(json_norm)
        
        if not real_path:
            # Debugowanie (pokazujemy to tylko 3 razy żeby nie spamować)
            if processed_count < 3: 
                print(f"⚠️ DEBUG BRAKU:")
                print(f"   JSON (czysty): '{json_norm}' (z '{json_filename}')")
                # Próbujemy znaleźć coś podobnego na dysku dla porównania
                similar = [k for k in list(disk_file_map.keys())[:500] if json_norm[:10] in k]
                if similar:
                     print(f"   Najbardziej podobny na dysku: '{similar[0]}'")
                else:
                     print(f"   Brak podobnych plików na dysku (sprawdzono pierwsze 500).")
            processed_count += 1
            continue

        matched_count += 1
        img_w = img_data['width']
        img_h = img_data['height']
        
        txt_path = os.path.splitext(real_path)[0] + ".txt"
        
        with open(txt_path, 'w') as out_f:
            for ann in anns:
                bbox = ann['bbox'] 
                x, y, w, h = bbox
                
                x_center = (x + w / 2) / img_w
                y_center = (y + h / 2) / img_h
                w_norm = w / img_w
                h_norm = h / img_h
                
                x_center = max(0, min(1, x_center))
                y_center = max(0, min(1, y_center))
                w_norm = max(0, min(1, w_norm))
                h_norm = max(0, min(1, h_norm))

                class_id = id_map[ann['category_id']]
                out_f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")
    
    print(f"\n--- PODSUMOWANIE ---")
    print(f"Liczba zdjęć w JSON: {len(annotations_by_image)}")
    print(f"✅ SUKCES: Utworzono pliki .txt dla: {matched_count} zdjęć")
    print(f"❌ BRAK: Nie dopasowano: {len(annotations_by_image) - matched_count}")

if __name__ == '__main__':
    convert_coco_to_yolo_smart()

In [ ]:
# 1. Wczytaj swój obecny najlepszy model (żeby kontynuować naukę, a nie zaczynać od zera)
# Upewnij się, że ścieżka do best.pt jest poprawna
model = YOLO('runs/detect/train4/weights/best.pt') 

# 2. Odpal dotrenowanie na wyczyszczonym zestawie (tylko zdjęcia ze statkami)
model.train(
    data=r"c:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\dataset_only_ships\data_only_ships.yaml",
    epochs=10,        # 50 epok spokojnie wystarczy na "doszlifowanie"
    imgsz=1280,
    batch=16,
    name='finetune_night_ships'  # Pod taką nazwą zapisze się wynik
)

In [ ]:
# --- KONFIGURACJA ---
source_root = Path(r"c:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\dataset_fixed")
dest_root = Path(r"c:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\dataset_split_70_20_10")
# --------------------

def split_and_filter_dataset():
    # 1. Czyszczenie folderu docelowego (żeby nie mieszać ze starymi próbami)
    if dest_root.exists():
        print("Usuwam stary folder docelowy, żeby zrobić czysty podział...")
        shutil.rmtree(dest_root)
    
    # 2. Zbieranie wszystkich poprawnych par (zdjęcie + label ze statkiem)
    print("Zbieranie danych z folderu źródłowego...")
    src_labels_dir = source_root / "labels"
    src_images_dir = source_root / "images"
    
    valid_pairs = [] # Lista tupli: (ścieżka_do_label, ścieżka_do_img)

    # Szukamy wszystkich plików .txt w źródle (rekurencyjnie, nieważne czy są w train/val/test)
    for label_file in src_labels_dir.rglob("*.txt"):
        
        # Sprawdzamy czy plik NIE jest pusty (czy ma statek)
        has_content = False
        with open(label_file, 'r') as f:
            if any(line.strip() for line in f):
                has_content = True
        
        if has_content:
            # Szukamy odpowiadającego zdjęcia
            # Musimy zgadnąć gdzie jest zdjęcie, bo struktura folderów images zazwyczaj odpowiada labels
            # Sztuczka: podmieniamy w ścieżce "labels" na "images" i szukamy pliku
            
            # Pobieramy relatywną ścieżkę pliku txt względem folderu labels
            # np. jeśli plik to .../labels/train/plik.txt, to relative to "train/plik.txt"
            rel_path = label_file.relative_to(src_labels_dir)
            
            image_found = None
            # Szukamy zdjęcia w analogicznym podfolderze w images
            for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
                possible_img_path = src_images_dir / rel_path.with_suffix(ext)
                if possible_img_path.exists():
                    image_found = possible_img_path
                    break
            
            if image_found:
                valid_pairs.append((label_file, image_found))

    print(f"Znaleziono łącznie {len(valid_pairs)} zdjęć ze statkami.")
    
    if len(valid_pairs) == 0:
        print("Błąd: Nie znaleziono żadnych danych! Sprawdź ścieżki.")
        return

    # 3. Mieszanie i dzielenie (70% / 20% / 10%)
    random.seed(42) # Żeby zawsze dzieliło tak samo
    random.shuffle(valid_pairs)

    total = len(valid_pairs)
    train_end = int(total * 0.70)
    val_end = int(total * 0.90) # 70% + 20% = 90%

    train_set = valid_pairs[:train_end]
    val_set = valid_pairs[train_end:val_end]
    test_set = valid_pairs[val_end:]

    print(f"Podział: Train: {len(train_set)}, Val: {len(val_set)}, Test: {len(test_set)}")

    # 4. Funkcja kopiująca
    def copy_files(dataset, split_name):
        dst_img_dir = dest_root / "images" / split_name
        dst_lbl_dir = dest_root / "labels" / split_name
        dst_img_dir.mkdir(parents=True, exist_ok=True)
        dst_lbl_dir.mkdir(parents=True, exist_ok=True)

        for lbl_src, img_src in dataset:
            shutil.copy2(lbl_src, dst_lbl_dir / lbl_src.name)
            shutil.copy2(img_src, dst_img_dir / img_src.name)

    # Kopiujemy fizycznie pliki
    print("Kopiowanie plików (to chwilę potrwa)...")
    copy_files(train_set, "train")
    copy_files(val_set, "val")
    copy_files(test_set, "test")

    # 5. Tworzenie YAML
    yaml_content = f"""
path: {dest_root.as_posix()}
train: images/train
val: images/val
test: images/test

names:
  0: ship
"""
    yaml_path = dest_root / "data_split.yaml"
    with open(yaml_path, "w") as f:
        f.write(yaml_content)

    print("-" * 30)
    print("GOTOWE!")
    print(f"Nowy dataset jest tu: {dest_root}")
    print(f"Plik YAML do treningu: {yaml_path}")

if __name__ == "__main__":
    split_and_filter_dataset()